In [4]:
// load project dll
#r "nuget: Moq"
#r "../../bin/Debug/net9.0/DiscordTaskBotV2.dll"

Installed Packages Moq, 4.20.72

In [ ]:
// dependencies
using Moq;
using System;
using System.Collections.Generic;
using System.Threading.Tasks;
using DiscordTaskBot.Core;

In [6]:
// configs and mocks
AppearanceConfig appearanceConfig = new();
BehaviorConfig behaviorConfig = new();

In [ ]:
public class DiscordMessageBuilderMock
{
    private readonly Mock<IDiscordMessageBuilder> _mock;
    private readonly Dictionary<ulong, string> _messageStorage;
    private ulong _nextMessageId = 1;
    private string _currentMessageContent = string.Empty;

    public DiscordMessageBuilderMock()
    {
        _messageStorage = new Dictionary<ulong, string>();
        _mock = new Mock<IDiscordMessageBuilder>();
        
        SetupMockMethods();
    }

    public Mock<IDiscordMessageBuilder> Mock => _mock;
    public IDiscordMessageBuilder Object => _mock.Object;

    private void SetupMockMethods()
    {
        // SetTitle
        _mock.Setup(m => m.SetTitle(It.IsAny<string>()))
            .Returns<string>(title =>
            {
                _currentMessageContent += $"[TITLE: {title}]\n";
                return _mock.Object;
            });

        // SetDescription
        _mock.Setup(m => m.SetDescription(It.IsAny<string>()))
            .Returns<string>(description =>
            {
                _currentMessageContent += $"[DESCRIPTION: {description}]\n";
                return _mock.Object;
            });

        // AddField
        _mock.Setup(m => m.AddField(It.IsAny<string>(), It.IsAny<string>(), It.IsAny<bool>()))
            .Returns<string, string, bool>((name, value, inline) =>
            {
                _currentMessageContent += $"[FIELD: {name} = {value} (inline: {inline})]\n";
                return _mock.Object;
            });

        // SetColor
        _mock.Setup(m => m.SetColor(It.IsAny<uint>()))
            .Returns<uint>(colorHex =>
            {
                _currentMessageContent += $"[COLOR: #{colorHex:X6}]\n";
                return _mock.Object;
            });

        // AddButton
        _mock.Setup(m => m.AddButton(It.IsAny<string>(), It.IsAny<string>(), It.IsAny<ButtonType>()))
            .Returns<string, string, ButtonType>((label, customId, buttonType) =>
            {
                _currentMessageContent += $"[BUTTON: {label} ({customId}) - {buttonType}]\n";
                return _mock.Object;
            });

        // AddFooter
        _mock.Setup(m => m.AddFooter(It.IsAny<string>()))
            .Returns<string>(label =>
            {
                _currentMessageContent += $"[FOOTER: {label}]\n";
                return _mock.Object;
            });

        // SendToChannel
        _mock.Setup(m => m.SendToChannel(It.IsAny<ulong>()))
            .Returns<ulong>(async channelId =>
            {
                var messageId = _nextMessageId++;
                var location = new TaskLocation(channelId, messageId);
                
                _messageStorage[messageId] = _currentMessageContent;
                
                Console.WriteLine($"=== WIADOMOŚĆ WYSŁANA DO KANAŁU {channelId} ===");
                Console.WriteLine(_currentMessageContent);
                Console.WriteLine($"=== KONIEC WIADOMOŚCI (ID: {messageId}) ===\n");
                
                _currentMessageContent = string.Empty; // Reset dla następnej wiadomości
                
                return await Task.FromResult(location);
            });

        // SendToUser
        _mock.Setup(m => m.SendToUser(It.IsAny<ulong>()))
            .Returns<ulong>(async userId =>
            {
                Console.WriteLine($"=== WIADOMOŚĆ WYSŁANA DO UŻYTKOWNIKA {userId} ===");
                Console.WriteLine(_currentMessageContent);
                Console.WriteLine($"=== KONIEC WIADOMOŚCI ===\n");
                
                _currentMessageContent = string.Empty;
                
                await Task.CompletedTask;
            });

        // EditExistingMessage
        _mock.Setup(m => m.EditExistingMessage(It.IsAny<TaskLocation>()))
            .Returns<TaskLocation>(async location =>
            {
                if (_messageStorage.ContainsKey(location.MessageID))
                {
                    _messageStorage[location.MessageID] = _currentMessageContent;
                    
                    Console.WriteLine($"=== WIADOMOŚĆ ZEDYTOWANA (ID: {location.MessageID}) ===");
                    Console.WriteLine(_currentMessageContent);
                    Console.WriteLine($"=== KONIEC ZEDYTOWANEJ WIADOMOŚCI ===\n");
                }
                else
                {
                    Console.WriteLine($"=== BRAK WIADOMOŚCI O ID: {location.MessageID} DO EDYCJI ===");
                }
                
                _currentMessageContent = string.Empty;
                
                await Task.CompletedTask;
            });
    }

    // Metody pomocnicze do testów
    public string GetMessageContent(ulong messageId)
    {
        return _messageStorage.TryGetValue(messageId, out var content) ? content : null;
    }

    public bool MessageExists(ulong messageId)
    {
        return _messageStorage.ContainsKey(messageId);
    }

    public void ClearStorage()
    {
        _messageStorage.Clear();
        _nextMessageId = 1;
        _currentMessageContent = string.Empty;
    }
}